In [3]:
import cvxpy as cp
import numpy as np

regions = ['R1', 'R2']
R = len(regions)
T = 4  
L = 2  
M = 1000  
demand = np.full((R, T), 15)
fixed_cost = 100
variable_cost = 1
holding_cost = 0.5
shortage_penalty = 10
expiration_penalty = 3

q = cp.Variable((R, T))                     
x = cp.Variable((R, T), boolean=True)      
I = cp.Variable((R, T, L))                 
S = cp.Variable((R, T))                     
E = cp.Variable((R, T))                   

constraints = []

for r in range(R):
    for t in range(T):
        constraints.append(q[r, t] <= M * x[r, t])
        constraints.append(E[r, t] == I[r, t, L-1])
        constraints.append(cp.sum(I[r, t, :]) + S[r, t] >= demand[r, t])
        for l in range(L):
            constraints.append(I[r, t, l] >= 0)
for r in range(R):
    for t in range(1, T):
        constraints.append(I[r, t, 0] == q[r, t-1])  
        for l in range(1, L):
            constraints.append(I[r, t, l] == I[r, t-1, l-1])  
for r in range(R):
    for l in range(L):
        constraints.append(I[r, 0, l] == 0)
constraints += [var >= 0 for var in [q, S, E]]
cost = 0
for r in range(R):
    for t in range(T):
        cost += fixed_cost * x[r, t] + variable_cost * q[r, t]
        cost += holding_cost * cp.sum(I[r, t, :])
        cost += shortage_penalty * S[r, t] + expiration_penalty * E[r, t]
problem = cp.Problem(cp.Minimize(cost), constraints)
problem.solve(solver=cp.GUROBI)
print(f"Total cost: {problem.value}")
print("Delivery quantities:\n", q.value)
print("Delivery activations:\n", x.value)
print("Expired inventory:\n", E.value)


Total cost: 895.0
Delivery quantities:
 [[15.  0. 15.  0.]
 [15.  0. 15.  0.]]
Delivery activations:
 [[ 1. -0.  1.  0.]
 [ 1.  0.  1.  0.]]
Expired inventory:
 [[-0.  0. 15.  0.]
 [-0.  0. 15.  0.]]
